# Librerías y funciones

## Librería

In [ ]:
# Se importan las librerías para el análisis de datos
import pandas as pd
import numpy as np
from scipy import stats

# Librerías para graficar
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
import plotly.graph_objects as go

# Para ML
from sklearn.cluster import KMeans

# Para hacer lectura de archivos json
import json

# Para revisar si los archivos existen en la dirección indicada
from os.path import exists

# Se define la paleta de colores de Dataket 8)
colors = ["#264653", "#2a9d8f", "#e9C46a", "#F4A261", "#E76F51"]

## Funciones

### Lectura de archivos

In [ ]:
# Se define una función para leer un json a partir de una direccion
def leer_json(direccion):

    # Se lee el archivo desde su dirección y se guarda en un diccionario
    with open(direccion,'r') as f:
        data = json.loads(f.read())
    return data

### Series de tiempo

In [ ]:
# Se crean un par de arreglos a partir de las variables  indicadas
def declaracion_personal(curp, variable_x="anio", variable_y="valor_neto"):
  # Se revisa que el curp exista en las declaraciones
  if len(df.loc[curp]["Declaraciones"])>0:
    # Se construyen las listas
    valores = []
    anios = []
    for i in range(len(df.loc[curp]["Declaraciones"])-1):
      anio = df_declaraciones.iloc[df.loc[curp]["Declaraciones"][i]][variable_x]
      valor = df_declaraciones.iloc[df.loc[curp]["Declaraciones"][i]][variable_y]
      
      anios.append(anio)
      valores.append(valor)
    return anios[:], valores[:]

In [ ]:
# Se crea la gráfica de un par de arreglos y se les hace un ajuste lineal
# por mínimos cuadrados y otro por el método de Theil-Sen
def grafica_serie_temporal(anio, valor, cadena):
  # Se revisa que el curp exista en las declaraciones
  if len(anio)==len(valor) and len(anio)>1:
    auxiliar = pd.DataFrame({'Anio': np.array(anio),
                             'Valor': np.array(valor)}).sort_values(by=['Anio']).reset_index()
    anio_minimo = auxiliar["Anio"].min()

    if len(auxiliar[auxiliar["Anio"]==anio_minimo])>1:
      auxiliar = auxiliar.drop(0)

    # Ajuste lineal
    ajuste = np.polyfit(auxiliar["Anio"]-anio_minimo,
                        auxiliar["Valor"], 1) # Mínimos cuadrados
    recta = np.poly1d(ajuste)

    theil = stats.theilslopes(auxiliar["Valor"],
                              auxiliar["Anio"]-anio_minimo, 0.10) # Regresión de Theil-Sen
    

    # Se grafica la serie de tiempo
    fig = px.line(auxiliar, x="Anio", y="Valor",
                    color_discrete_sequence=[colors[1]],
                    title=f"Serie temporal del valor de {cadena}",
                    labels={"Anio": "Año", "Valor": f"Valor de {cadena} (MXN)"})
    
    # Se grafica la recta por mínimos cuadrados
    if (auxiliar["Valor"].diff()!=0).sum()>1:# Se evita que el ajuste no esté definido
      fig.add_trace(go.Line(x=auxiliar["Anio"],
                    y=recta(auxiliar["Anio"]-anio_minimo),
                    showlegend=False,
                    line=dict(color=colors[0], width=3)))
    # Se grafica la recta de Theil-Sen
    fig.add_trace(go.Line(x=auxiliar["Anio"],
                  y=theil[1]+theil[0]*(auxiliar["Anio"]-anio_minimo),
                  showlegend=False,
                  line=dict(color=colors[2], width=3)))

    # Se ajusta el tamaño de la letra
    fig.update_layout(font_size=15)

    # Se guarda el html para poder usarlo en la página web de Dataket 8)
    plotly.offline.plot(fig, filename='/content/drive/MyDrive/Dataton_2021/scatter_'+cadena.replace(" ", "")+'.html')

    # Se muestra la imagen
    fig.show()

### Para realizar muestras para el análisis de las variaciones

In [ ]:
# Se realiza una muestra para hacer comparaciones entre las varianzas
def declaracion_muestra(df_sobre_declaraciones, variable_x, variable_y):
  lista = []
  for curp in set(df_sobre_declaraciones["curp"]):

    anio, valor = declaracion_personal(curp, variable_x, variable_y)

    # Se eliminan los datos que repiten año
    auxiliar = pd.DataFrame({'Anio': np.array(anio),
                             'Valor': np.array(valor)}).sort_values(by=['Anio']).reset_index()
    anio_minimo = auxiliar["Anio"].min()
    if len(auxiliar[auxiliar["Anio"]==anio_minimo])>1:
      auxiliar = auxiliar.drop(0).reset_index()

    # Aquí se realiza la revisión de la métrica a usar
    if len(valor)>2:
      suma = 0
      for i in range(len(auxiliar["Valor"])-1):
        if auxiliar["Anio"][i+1]!=auxiliar["Anio"][i]:
          suma += abs((auxiliar["Valor"][i+1]-auxiliar["Valor"][i])/(auxiliar["Anio"][i+1]-auxiliar["Anio"][i]))
      if auxiliar["Valor"].mean()!=0:
        lista.append(abs(suma/auxiliar["Valor"].mean()))

  return np.array(lista[:])

### Agrupación por K-Means y Pairplot

In [ ]:
def agrupacion(data_x, n_clusters):
  # Se prepara el algoritmo K-Means y el PCA
  kmeans = KMeans(n_clusters=n_clusters, n_init=10, random_state=0, max_iter=10_000)

  # Se preparan los arrays
  X = data_x.copy()

  # Se ajusta el modelo de k-means minimizando el efecto de
  # los outliers al filtrarlos
  kmeansclus = kmeans.fit(X[X["valor_neto"].between(X["valor_neto"].quantile(.05),
                                                    X["valor_neto"].quantile(.95))])
    
  # Se hacen predicciones de los grupos
  y_kmeans = kmeansclus.predict(X)
  X["grupo"] = y_kmeans

  # Se grafica el pairplot
  sns.pairplot(hue="grupo", data=X)

### El p-value de una variable

In [ ]:
# Se calcula el p-value del registro en la distribución de la
# variable dada
def declaracion_pvalue(curp, variable):
  if curp in set(df_curp["curp"]):
    columna = df_curp[variable]
    registro = df_curp[df_curp["curp"]==curp][variable].iloc[0]
    # Se promedia el número de registros de la muestra superiores
    # al estadístico de prueba
    return len(columna[columna<registro])/len(columna), len(columna[columna>registro])/len(columna)

## Lectura de archivos

In [ ]:
# Se define el url donde se encuentra el archivo (en el repositorio de Github)
url_declaraciones = "/content/drive/MyDrive/Dataton_2021/declaraciones.json"

# Se lee el archivo
declaraciones = leer_json(url_declaraciones)

In [ ]:
# Se define el url donde se encuentra el archivo (en el repositorio de Github)
url_contrataciones = "/content/drive/MyDrive/Dataton_2021/SistemaS2.json"

# Se lee el archivo
contratos = leer_json(url_contrataciones)

In [ ]:
# Se define el url donde se encuentra el archivo (en el repositorio de Github)
url_sanciones = "/content/drive/MyDrive/Dataton_2021/Sistema3Servidores.json"

# Se lee el archivo
multas = leer_json(url_sanciones)

### Creación del binario para las consultas

In [ ]:
if not exists('/content/drive/MyDrive/Dataton_2021/relations.pkl'):
    # Obtenemos curps individuales (numero de servidores públicos)

    curps = []

    for multa in multas:
        curps.append(multa['servidorPublicoSancionado']['curp'])

    for contrato in contratos: 
        curps.append(contrato['curp'])

    for declaracion in declaraciones:
        curps.append(declaracion['declaracion']['situacionPatrimonial']['datosGenerales']['curp'])

    curps = set(curps)
    
    print(f"Tenemos registros de {len(curps)} personas.")
    
    # Dataframe para indices por persona.

    aux = []
    for i in range(len(curps)):
        aux.append([[],[],[]])

    df = pd.DataFrame(aux, index = curps, columns = ['Multas', 'Declaraciones', 'Contratos'])

    for curp in curps:
        for i, multa in enumerate(multas):
            if multa['servidorPublicoSancionado']['curp'] == curp:
                df.at[curp, 'Multas'].append(i)

        for i, declaracion in enumerate(declaraciones):
            if declaracion['declaracion']['situacionPatrimonial']['datosGenerales']['curp'] == curp:
                df.at[curp, 'Declaraciones'].append(i)

        for i, contrato in enumerate(contratos): 
            if contrato['curp'] == curp:
                df.at[curp, 'Contratos'].append(i)
                
    df.to_pickle('/content/drive/MyDrive/Dataton_2021/relations.pkl')

else: 
    df = pd.read_pickle('/content/drive/MyDrive/Dataton_2021/relations.pkl')

### Las columnas más importantes de las Declaraciones

In [ ]:
if not exists('/content/drive/MyDrive/Dataton_2021/tabla.pkl'):

  # Se obtienen las columnas más importantes de las Declaraciones para facilitar
  # las consultas
  tabla = []
  for i in range(len(declaraciones)):
    # En esta lista se guardaran los datos relevantes del registro actual
    lista = []

    # id
    lista.append(declaraciones[i]["id"])
    # Año
    lista.append(int(declaraciones[i]["metadata"]["actualizacion"][:4]))
    # CURP
    lista.append(declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosGenerales"]["curp"])

    # Nombre y apellidos
    lista.append(declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosGenerales"]["nombre"])
    lista.append(declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosGenerales"]["primerApellido"])
    lista.append(declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosGenerales"]["segundoApellido"])

    # Nivel de escolaridad
    lista.append(declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosCurricularesDeclarante"]["escolaridad"][-1]["nivel"]["clave"])
    # Nivel de orden de gobierno
    lista.append(declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosEmpleoCargoComision"]["nivelOrdenGobierno"])
    # Nombre del ente público
    lista.append(declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosEmpleoCargoComision"]["nombreEntePublico"])

    # Tiene pareja
    try:
      try:
        if not declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosPareja"]["nunguno"]: # Ojo: dice "nunguno" en lugar de "ninguno"
          lista.append(1) # Sí tiene pareja
        else:
          lista.append(0) # No tiene pareja
      except:
        if not declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosPareja"]["ninguno"]: # Ojo: dice "nunguno" en lugar de "ninguno"
          lista.append(1) # Sí tiene pareja
        else:
          lista.append(0) # No tiene pareja
    except:
      lista.append(0)
    
    # Número de dependientes económicos
    try:
      try:
        if not declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosDependienteEconomico"]["nunguno"]: # Ojo: dice "nunguno" en lugar de "ninguno"
          lista.append(len(declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosDependienteEconomico"]["dependienteEconomico"])) # Sí tiene dependientes
        else:
          lista.append(0) # No tiene dependientes
      except:
        if not declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosDependienteEconomico"]["ninguno"]: # Ojo: dice "nunguno" en lugar de "ninguno"
          lista.append(len(declaraciones[i]["declaracion"]["situacionPatrimonial"]["datosDependienteEconomico"]["dependienteEconomico"])) # Sí tiene dependientes
        else:
          lista.append(0)
    except:
      lista.append(0)

    # Remuneración del cargo público
    try:
      valor = declaraciones[i]["declaracion"]["situacionPatrimonial"]["ingresos"]["remuneracionMensualCargoPublico"]["valor"]
      lista.append(12*valor)
    except:
      try:
        valor = declaraciones[i]["declaracion"]["situacionPatrimonial"]["ingresos"]["remuneracionConclusionCargoPublico"]["valor"]
        lista.append(valor)   
      except:
        try:
          valor = declaraciones[i]["declaracion"]["situacionPatrimonial"]["ingresos"]["remuneracionAnualCargoPublico"]["valor"]
          lista.append(valor)   
        except:
          lista.append(0)

    # Ingreso anual promedio
    try:
      valor = declaraciones[i]["declaracion"]["situacionPatrimonial"]["ingresos"]["totalIngresosMensualesNetos"]["valor"]
      lista.append(12*valor)
    except:
      try:
        valor = declaraciones[i]["declaracion"]["situacionPatrimonial"]["ingresos"]["totalIngresosAnualesNetos"]["valor"]
        lista.append(valor)   
      except:
        try:
          valor = declaraciones[i]["declaracion"]["situacionPatrimonial"]["ingresos"]["ingresoAnualNetoDeclarante"]["valor"]
          lista.append(valor)   
        except:
          valor = declaraciones[i]["declaracion"]["situacionPatrimonial"]["ingresos"]["totalIngresosConclusionNetos"]["valor"]
          lista.append(valor) 
    
    # Valor total de bienes inmuebles.
    try:
      if not declaraciones[i]["declaracion"]["situacionPatrimonial"]["bienesInmuebles"]["ninguno"]:
        propiedades = 0
        for propiedad in declaraciones[i]["declaracion"]["situacionPatrimonial"]["bienesInmuebles"]["bienInmueble"]:
          try:
            propiedades += propiedad["valorAdquisicion"]["valor"]
          except:
            propiedades += propiedad["valorAdquisicion"]
        lista.append(propiedades)
      else:
        lista.append(0)
    except:
      lista.append(0)

    # Valor total de adquisición de vehículos.
    try:
      if not declaraciones[i]["declaracion"]["situacionPatrimonial"]["vehiculos"]["ninguno"]:
        propiedades = 0
        for propiedad in declaraciones[i]["declaracion"]["situacionPatrimonial"]["vehiculos"]["vehiculo"]:
          try:
            propiedades += propiedad["valorAdquisicion"]["valor"]
          except:
            propiedades += propiedad["valorAdquisicion"]
        lista.append(propiedades)
      else:
        lista.append(0)
    except:
      lista.append(0)

    # Valor total de bienes muebles
    try:
      if not declaraciones[i]["declaracion"]["situacionPatrimonial"]["bienesMuebles"]["ninguno"]:
        propiedades = 0
        for propiedad in declaraciones[i]["declaracion"]["situacionPatrimonial"]["bienesMuebles"]["bienMueble"]:
          try:
            propiedades += propiedad["valorAdquisicion"]["valor"]
          except:
            propiedades += propiedad["valorAdquisicion"]
        lista.append(propiedades)
      else:
        lista.append(0)
    except:
      lista.append(0)

    # Valor total de inversiones (saldo diciembre anterior*rendimiento).
    try:
      if not declaraciones[i]["declaracion"]["situacionPatrimonial"]["inversiones"]["ninguno"]:
        propiedades = 0
        for propiedad in declaraciones[i]["declaracion"]["situacionPatrimonial"]["inversiones"]["inversion"]:
          try:
            propiedades += (1+int(propiedad["porcentajeIncrementoDecremento"][:-1])/100)*propiedad["saldoDiciembreAnterior"]["valor"]
          except:
            try:
              propiedades += propiedad["saldoSituacionActual"]["valor"]
            except:
              propiedades += propiedad["saldoFechaConclusion"]["valor"]
        lista.append(propiedades)
      else:
        lista.append(0)
    except:
      lista.append(0)

    # Valor total de deudas (saldo diciembre anterior*rendimiento)
    try:
      if not declaraciones[i]["declaracion"]["situacionPatrimonial"]["adeudos"]["ninguno"]:
        propiedades = 0
        for propiedad in declaraciones[i]["declaracion"]["situacionPatrimonial"]["adeudos"]["adeudo"]:
          try:
            variable = (1+int(propiedad["porcentajeIncrementoDecremento"][:-1])/100)*propiedad["saldoInsolutoDiciembreAnterior"]["valor"]
            propiedades += variable
          except:
            try:
              variable = propiedad["saldoInsolutoSituacionActual"]["valor"]
              propiedades += variable
            except:
              variable = propiedad["saldoInsolutoFechaConclusion"]["valor"]
              propiedades += variable
        
        lista.append(propiedades)
      else:
        lista.append(0)
    except:
      lista.append(0)

    # Valor de propiedades
    lista.append(lista[-3]+lista[-4]+lista[-5])

    # Valor con inversiones
    lista.append(lista[-1]+lista[-3])

    # Valor neto (Valor con inversiones menos deudas)
    lista.append(lista[-1]-lista[-3])

    # Ha tenido sanciones
    if len(df.loc[lista[2]]["Multas"])==0:
      lista.append(0) # No se le ha sancionado
    else:
      lista.append(1) # Se le ha sancionado

    # Se guarda el registro
    tabla.append(lista)

  df_declaraciones = pd.DataFrame(tabla, columns=["id", "anio", "curp", "nombre", "primerApellido",
                              "segundoApellido", "nivelEducativo",
                              "ordenGobierno", "entePublico", "pareja",
                              "numeroDependientes", "remuneracionCargo",
                              "ingresosAnuales", "bienesInmuebles",
                              "vehiculos", "bienesMuebles", "inversiones",
                              "adeudos", "propiedades", "propiedades_inversiones",
                              "valor_neto", "multas"])
  
  df_declaraciones.to_pickle('/content/drive/MyDrive/Dataton_2021/tabla.pkl')

else: 
  df_declaraciones = pd.read_pickle('/content/drive/MyDrive/Dataton_2021/tabla.pkl')

### Datos agrupados por CURP

In [ ]:
if not exists('/content/drive/MyDrive/Dataton_2021/curp.pkl'):

  # Se agrupan las columnas más importantes de las Declaraciones para facilitar
  # las consultas por persona

  tabla = []
  for curp in df.index:
    if curp in set(df_declaraciones["curp"]):
      # Los registros a analizar
      registros = df_declaraciones[df_declaraciones["curp"]==curp].sort_values(by=['anio']).reset_index()
      anio_minimo = registros["anio"].min()
      if len(registros[registros["anio"]==anio_minimo])>1:
        registros = registros.drop(0).reset_index()
      numero = len(registros)

      # curp
      lista = [curp]

      # Nivel educativo promedio
      sublista = []
      for nivel in registros["nivelEducativo"]:
        if nivel=="LIC":
          sublista.append(1)
        elif nivel=="ESP":
          sublista.append(2)
        elif nivel=="MAE":
          sublista.append(3)
        else:
          sublista.append(4)
      lista.append(np.array(sublista).mean())

      # Orden de gobierno promedio
      sublista = []
      for nivel in registros["ordenGobierno"]:
        if nivel=="MUNICIPAL_ALCALDIA":
          sublista.append(1)
        elif nivel=="ESTATAL":
          sublista.append(2)
        else:
          sublista.append(3)
      lista.append(np.array(sublista).mean())

      # Número de pareja promedio
      sublista = registros["pareja"]
      lista.append(np.array(sublista).mean())

      # Número de dependientes promedio
      sublista = registros["numeroDependientes"]
      lista.append(np.array(sublista).mean())

      if numero>=3:
        # Variación de los ingresos anuales
        sublista = registros["ingresosAnuales"].iloc[:]
        lista.append(sublista.std()/sublista.mean())
        subvalor = np.array(sublista).mean()
        # Remuneración del cargo más reciente normalizada al ingreso anual promedio
        lista.append(registros["remuneracionCargo"].iloc[-1]/subvalor)
      else:
        # Variación de los ingresos anuales
        subvalor = registros["ingresosAnuales"].iloc[-1]
        lista.append(0)
        # Remuneración del cargo más reciente normalizada al ingreso anual promedio
        lista.append(registros["remuneracionCargo"].iloc[-1]/subvalor)

      # Variación estandarizada del valor de los bienes inmuebles
      try:
        sublista = registros["bienesInmuebles"].iloc[:]
        if len(sublista)>2:
          lista.append(sublista.std()/sublista.mean())
        else:
          lista.append(0)
      except:
        lista.append(0)

      # Variación estandarizada del valor de los vehículos
      try:
        sublista = registros["vehiculos"].iloc[:]
        if len(sublista)>2:
          lista.append(sublista.std()/sublista.mean())
        else:
          lista.append(0)
      except:
        lista.append(0)

      # Variación estandarizada del valor de los bienes muebles
      try:
        sublista = registros["bienesMuebles"].iloc[:]
        if len(sublista)>2:
          lista.append(sublista.std()/sublista.mean())
        else:
          lista.append(0)
      except:
        lista.append(0)

      # Variación estandarizada del valor de las inversiones
      try:
        sublista = registros["inversiones"].iloc[:]
        if len(sublista)>2:
          lista.append(sublista.std()/sublista.mean())
        else:
          lista.append(0)
      except:
        lista.append(0)
      
      # Variación estandarizada del valor de los adeudos
      try:
        sublista = registros["adeudos"].iloc[:]
        if len(sublista)>2:
          lista.append(sublista.std()/sublista.mean())
        else:
          lista.append(0)
      except:
        lista.append(0)
      
      # Variación estandarizada del valor de las propiedades
      try:
        sublista = registros["propiedades"].iloc[:]
        if len(sublista)>2:
          lista.append(sublista.std()/sublista.mean())
        else:
          lista.append(0)
      except:
        lista.append(0)

      # Variación estandarizada del valor de las propiedades e inversiones
      try:
        sublista = registros["propiedades_inversiones"].iloc[:]
        if len(sublista)>2:
          lista.append(sublista.std()/sublista.mean())
        else:
          lista.append(0)
      except:
        lista.append(0)

      # Variación estandarizada del valor de las propiedades e inversiones menos adeudos
      try:
        sublista = registros["valor_neto"].iloc[:]
        if len(sublista)>2:
          lista.append(sublista.std()/sublista.mean())
        else:
          lista.append(0)
      except:
        lista.append(0)
      
      # Multas
      lista.append(registros["multas"].iloc[0])
      
      # Se guarda el registro
      tabla.append(lista)

  df_curp = pd.DataFrame(tabla, columns=["curp", "nivelEducativo",
                              "ordenGobierno", "pareja",
                              "numeroDependientes", "ingresosAnuales",
                              "remuneracionCargo", "bienesInmuebles",
                              "vehiculos", "bienesMuebles", "inversiones",
                              "adeudos", "propiedades", "propiedades_inversiones",
                              "valor_neto", "multas"])

  df_curp.to_pickle("/content/drive/MyDrive/Dataton_2021/curp.pkl")

else:
  df_curp = pd.read_pickle("/content/drive/MyDrive/Dataton_2021/curp.pkl")

### Outliers

In [ ]:
if not exists("/content/drive/MyDrive/Dataton_2021/interesantes.pkl"):
  # Se crea la lista de outliers al nivel de confianza de 99.9%
  lista = []
  for curp in set(df_curp["curp"]):
    interesante = False
    columnas = []

    registro = df_declaraciones[df_declaraciones["curp"]==curp]
    # Se revisan las columnas con datos patrimoniales y de ingresos
    for columna in list(df_curp.columns)[5:15]:
      if declaracion_pvalue(curp, columna)[0]>=0.9 and declaracion_pvalue(curp, columna)[1]<=0.001:
        subregistro = registro[columna]
        no_es_constante = (subregistro.diff()!=0).sum()>2
        if columna!="remuneracionCargo" and no_es_constante:
          interesante = True
          columnas.append(columna)
        elif columna=="remuneracionCargo":
          interesante = True
          columnas.append(columna)

    if interesante:
      lista.append([curp, len(columnas), columnas])

  df_interesantes = pd.DataFrame(lista,
                                columns=["curp", "numero",
                                          "columnas_interesantes"]).sort_values(by=["numero"],
                                                                                ascending=False).reset_index(drop=True)
  df_interesantes.to_pickle("/content/drive/MyDrive/Dataton_2021/interesantes.pkl")

else:
  df_interesantes = pd.read_pickle("/content/drive/MyDrive/Dataton_2021/interesantes.pkl")

# Prototipo de Análisis Inicial

In [ ]:
curp = input("Ingrese el CURP a consultar: ")

# Nivel de Urgencia del CURP
presentacion = "\nEste CURP tiene un Nivel de Urgencia"
registro = int(df_interesantes[df_interesantes["curp"]==curp]["numero"])

if curp not in set(df_interesantes["curp"]):
  print(f"{presentacion} BAJO (color VERDE de semáforo)\n\n")
elif registro<=2:
  print(f"{presentacion} MEDIO (color AMARILLO de semáforo)\n\n")
elif registro<=4:
  print(f"{presentacion} ALTO (color NARANJA de semáforo)\n\n")
else:
  print(f"{presentacion} CRITICO (color ROJO de semáforo)\n\n")

# Las gráficas críticas
registro = list(df_interesantes[df_interesantes["curp"]==curp]["columnas_interesantes"])[0]
for columna in registro:
  auxiliar_anio, auxiliar_valor = declaracion_personal(curp, "anio", columna)
  grafica_serie_temporal(auxiliar_anio[:], auxiliar_valor[:], columna)

Ingrese el CURP a consultar: MABG880226MHGRLR03

Este CURP tiene un Nivel de Urgencia ALTO (color NARANJA de semáforo)




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


